In [1]:
import json
import re
import sentencepiece as spm

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
from prepro_utils import preprocess_text, encode_ids, encode_pieces

sp_model = spm.SentencePieceProcessor()
sp_model.Load('bert-base/sp10m.cased.v4.model')

with open('bert-base/sp10m.cased.v4.vocab') as fopen:
    v = fopen.read().split('\n')[:-1]
v = [i.split('\t') for i in v]
v = {i[0]: i[1] for i in v}

class Tokenizer:
    def __init__(self, v):
        self.vocab = v
        pass
    
    def tokenize(self, string):
        return encode_pieces(sp_model, string, return_unicode=False, sample=False)
    
    def convert_tokens_to_ids(self, tokens):
        return [sp_model.PieceToId(piece) for piece in tokens]
    
    def convert_ids_to_tokens(self, ids):
        return [sp_model.IdToPiece(i) for i in ids]
    
tokenizer = Tokenizer(v)

In [9]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import json
import tensorflow as tf
import itertools
from unidecode import unidecode
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re

In [5]:
BERT_INIT_CHKPNT = 'bert-base/model.ckpt'
BERT_CONFIG = 'bert-base/bert_config.json'

In [6]:
def cleaning(string):
    string = unidecode(string)
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', string)
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

In [10]:
df = pd.read_csv('../Malaya-Dataset/news-sentiment/sentiment-data-v2.csv')
Y = LabelEncoder().fit_transform(df.label)

with open('../Malaya-Dataset/polarity/polarity-negative-translated.txt','r') as fopen:
    texts = fopen.read().split('\n')
labels = [0] * len(texts)

with open('../Malaya-Dataset/polarity/polarity-positive-translated.txt','r') as fopen:
    positive_texts = fopen.read().split('\n')
labels += [1] * len(positive_texts)
texts += positive_texts
texts += df.iloc[:,1].tolist()
labels += Y.tolist()

assert len(labels) == len(texts)

In [11]:
import json
with open('../Malaya-Dataset/multidomain-sentiment/bm-amazon.json') as fopen:
    amazon = json.load(fopen)
    
with open('../Malaya-Dataset/multidomain-sentiment/bm-imdb.json') as fopen:
    imdb = json.load(fopen)
    
with open('../Malaya-Dataset/multidomain-sentiment/bm-yelp.json') as fopen:
    yelp = json.load(fopen)
    
texts += amazon['negative']
labels += [0] * len(amazon['negative'])
texts += amazon['positive']
labels += [1] * len(amazon['positive'])

texts += imdb['negative']
labels += [0] * len(imdb['negative'])
texts += imdb['positive']
labels += [1] * len(imdb['positive'])

texts += yelp['negative']
labels += [0] * len(yelp['negative'])
texts += yelp['positive']
labels += [1] * len(yelp['positive'])

In [14]:
import os
for i in [i for i in os.listdir('../Malaya-Dataset/twitter-sentiment/negative') if 'Store' not in i]:
    with open('../Malaya-Dataset/twitter-sentiment/negative/'+i) as fopen:
        a = json.load(fopen)
        texts += a
        labels += [0] * len(a)

In [15]:
for i in [i for i in os.listdir('../Malaya-Dataset/twitter-sentiment/positive') if 'Store' not in i]:
    with open('../Malaya-Dataset/twitter-sentiment/positive/'+i) as fopen:
        a = json.load(fopen)
        texts += a
        labels += [1] * len(a)

In [16]:
with open('../Malaya-Dataset/emotion/happy-twitter-malaysia.json') as fopen:
    a = json.load(fopen)
    texts += a
    labels += [1] * len(a)

In [17]:
with open('../Malaya-Dataset/emotion/anger-twitter-malaysia.json') as fopen:
    a = json.load(fopen)
    texts += a
    labels += [0] * len(a)

In [18]:
from tqdm import tqdm

for i in tqdm(range(len(texts))):
    texts[i] = cleaning(texts[i])

100%|██████████| 775692/775692 [00:10<00:00, 75982.63it/s]


In [19]:
tokenizer.tokenize(texts[1])

['▁kanak',
 '-',
 'kanak',
 '▁lelaki',
 '▁remaja',
 '▁yang',
 '▁begitu',
 '▁muda',
 '▁dan',
 '▁muda',
 '▁hanya',
 '▁dapat',
 '▁mene',
 'ka',
 '▁dengan',
 '▁lucu']

In [20]:
list(v.keys())[:10]

['<unk>',
 '<s>',
 '</s>',
 '<cls>',
 '<sep>',
 '<pad>',
 '<mask>',
 '<eod>',
 '<eop>',
 '.']

In [21]:
np.unique(labels, return_counts = True)

(array([0, 1]), array([404323, 371369]))

In [22]:
from tqdm import tqdm

input_ids = []

for text in tqdm(texts):
    tokens_a = tokenizer.tokenize(text)
    tokens = ["<cls>"] + tokens_a + ["<sep>"]
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    
    input_ids.append(input_id)

100%|██████████| 775692/775692 [01:01<00:00, 12698.67it/s]


In [23]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [24]:
epoch = 10
batch_size = 60
warmup_proportion = 0.1
num_train_steps = int(len(texts) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [25]:
bert_config.hidden_size

768

In [26]:
def create_initializer(initializer_range=0.02):
    return tf.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=training,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        output_layer = tf.layers.dense(
            output_layer,
            bert_config.hidden_size,
            activation=tf.tanh,
            kernel_initializer=create_initializer())
        self.logits_seq = tf.layers.dense(output_layer, dimension_output,
                                         kernel_initializer=create_initializer())
        self.logits_seq = tf.identity(self.logits_seq, name = 'logits_seq')
        self.logits = self.logits_seq[:, 0]
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [27]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from bert-base/model.ckpt


In [28]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_Y, test_Y = train_test_split(
    input_ids, labels, test_size = 0.2
)

In [29]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [30]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
        
    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/10343 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.837106
time taken: 2481.1744101047516
epoch: 0, training loss: 0.404100, training acc: 0.809305, valid loss: 0.358235, valid acc: 0.837106



train minibatch loop:   0%|          | 0/10343 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.837106, current acc: 0.845529
time taken: 2480.3709177970886
epoch: 1, training loss: 0.344728, training acc: 0.844731, valid loss: 0.345473, valid acc: 0.845529



train minibatch loop:   0%|          | 0/10343 [00:00<?, ?it/s]

time taken: 2482.0471045970917
epoch: 2, training loss: 0.301567, training acc: 0.868890, valid loss: 0.354789, valid acc: 0.844002



train minibatch loop:   0%|          | 0/10343 [00:00<?, ?it/s]

time taken: 2480.690477848053
epoch: 3, training loss: 0.258567, training acc: 0.891788, valid loss: 0.400294, valid acc: 0.840338



test minibatch loop: 100%|██████████| 2586/2586 [02:56<00:00, 14.62it/s, accuracy=0.846, cost=0.306] 

time taken: 2482.2882101535797
epoch: 4, training loss: 0.217496, training acc: 0.912215, valid loss: 0.445828, valid acc: 0.838391

break epoch:5



In [31]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'bert-base-sentiment/model.ckpt')

'bert-base-sentiment/model.ckpt'

In [32]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate,
    training = False
)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'bert-base-sentiment/model.ckpt')

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:Restoring parameters from bert-base-sentiment/model.ckpt


In [33]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'bert/embeddings/word_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/Softmax',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/output/dense/kernel',
 'bert/encoder/layer_0/output/dense/bias',
 'bert/encoder/layer_0/output/LayerNorm/gamma',
 'bert/encoder/layer_1/attention/self/query/kernel',
 

In [34]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_input_ids))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = test_Y[i: index]
    
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
            model.X: batch_x,
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 2586/2586 [02:39<00:00, 16.16it/s]


In [35]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['negative', 'positive'],digits = 5
    )
)

              precision    recall  f1-score   support

    negative    0.82923   0.87643   0.85218     80965
    positive    0.85618   0.80299   0.82873     74174

    accuracy                        0.84132    155139
   macro avg    0.84271   0.83971   0.84046    155139
weighted avg    0.84212   0.84132   0.84097    155139



In [36]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [37]:
freeze_graph('bert-base-sentiment', strings)

INFO:tensorflow:Restoring parameters from bert-base-sentiment/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 203 variables.
INFO:tensorflow:Converted 203 variables to const ops.
6954 ops in the final graph.


In [38]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [39]:
g = load_graph('bert-base-sentiment/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)
result = test_sess.run(tf.nn.softmax(logits), feed_dict = {x: [input_id]})
result

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


array([[9.9973792e-01, 2.6203212e-04]], dtype=float32)

In [40]:
import boto3

bucketName = 'huseinhouse-storage'
Key = 'bert-base-sentiment/frozen_model.pb'
outPutname = "v30/sentiment/bert-base-sentiment.pb"

s3 = boto3.client('s3',
                 aws_access_key_id='',
                 aws_secret_access_key='')

s3.upload_file(Key,bucketName,outPutname)